In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
import kts
from kts import *

In [3]:
df = pd.read_csv('../input/train.csv')

In [33]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [5]:
y = df['Survived'].values

In [6]:
@test
def drop_target_set_index(df):
    return df.copy().set_index('PassengerId').drop(['Survived'], axis=1)

drop_target_set_index(df)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [7]:
@register
def drop_target_set_index(df):
    return df.copy().set_index('PassengerId').drop(['Survived'], axis=1)

In [17]:
deregister(drop_name, force=True)

removing ../storage/features/drop_name.py


In [9]:
@register
def drop_name(df):
    return drop_target_set_index(df).drop(['Name'], axis=1)

In [9]:
drop_name(df).head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,3,male,22.0,1,0,A/5 21171,7.2500,None,S
2,1,female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
4,1,female,35.0,1,0,113803,53.1000,C123,S
5,3,male,35.0,0,0,373450,8.0500,None,S


In [10]:
@register
def ohe_sex(df):
    tmp = drop_name(df)
    tmp['Sex_ohe_male'] = (tmp['Sex'] == 'male')
    tmp['Sex_ohe_female'] = (tmp['Sex'] == 'female')
    return tmp[['Sex_ohe_male', 'Sex_ohe_female']].astype(int)

In [11]:
@register
def ohe_cabin_embarked(df):
    tmp = drop_name(df)
    res = pd.get_dummies(tmp, prefix_sep='_ohe_', columns=['Cabin', 'Embarked'])
    return res[[column for column in res.columns.values if 'ohe' in column]]

In [23]:
@register
def drop_ticket_cabin_embarked(df):
    tmp = drop_name(df)
    return tmp.drop(['Ticket', 'Cabin', 'Embarked'], axis=1)

In [27]:
@register
def drop_sex(df):
    tmp = drop_ticket_cabin_embarked(df)
    return tmp.drop(['Sex'], axis=1)

In [24]:
kts.storage.feature_constructors

[drop_ticket_cabin_embarked, drop_name, ohe_cabin_embarked, ohe_sex, drop_target_set_index]

In [41]:
from kts.feature.storage import FeatureSet

In [42]:
featureset = FeatureSet([drop_sex, ohe_sex, ohe_cabin_embarked], df_input=df)

In [50]:
featureset[[1, 2, 3, 4]]
featureset[[0]]
featureset[102:104]

,Pclass,Age,SibSp,Parch,Fare,Sex_ohe_male,Sex_ohe_female,Cabin_ohe_A10,Cabin_ohe_A14,Cabin_ohe_A16,...,Cabin_ohe_F G73,Cabin_ohe_F2,Cabin_ohe_F33,Cabin_ohe_F38,Cabin_ohe_F4,Cabin_ohe_G6,Cabin_ohe_T,Embarked_ohe_C,Embarked_ohe_Q,Embarked_ohe_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
2,1,38.0,1,0,71.2833,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,26.0,0,0,7.9250,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,35.0,1,0,53.1000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,3,35.0,0,0,8.0500,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,Pclass,Age,SibSp,Parch,Fare,Sex_ohe_male,Sex_ohe_female,Cabin_ohe_A10,Cabin_ohe_A14,Cabin_ohe_A16,...,Cabin_ohe_F G73,Cabin_ohe_F2,Cabin_ohe_F33,Cabin_ohe_F38,Cabin_ohe_F4,Cabin_ohe_G6,Cabin_ohe_T,Embarked_ohe_C,Embarked_ohe_Q,Embarked_ohe_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,3,22.0,1,0,7.25,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


,Pclass,Age,SibSp,Parch,Fare,Sex_ohe_male,Sex_ohe_female,Cabin_ohe_A10,Cabin_ohe_A14,Cabin_ohe_A16,...,Cabin_ohe_F G73,Cabin_ohe_F2,Cabin_ohe_F33,Cabin_ohe_F38,Cabin_ohe_F4,Cabin_ohe_G6,Cabin_ohe_T,Embarked_ohe_C,Embarked_ohe_Q,Embarked_ohe_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
103,1,21.0,0,1,77.2875,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
104,3,33.0,0,0,8.6542,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [44]:
df_test = pd.read_csv('../input/test.csv')
df_test['Survived'] = 0

In [45]:
featureset(df_test)

,Pclass,Age,SibSp,Parch,Fare,Sex_ohe_male,Sex_ohe_female,Cabin_ohe_A11,Cabin_ohe_A18,Cabin_ohe_A21,...,Cabin_ohe_F E46,Cabin_ohe_F E57,Cabin_ohe_F G63,Cabin_ohe_F2,Cabin_ohe_F33,Cabin_ohe_F4,Cabin_ohe_G6,Embarked_ohe_C,Embarked_ohe_Q,Embarked_ohe_S
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,3,34.5,0,0,7.8292,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
893,3,47.0,1,0,7.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
894,2,62.0,0,0,9.6875,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
895,3,27.0,0,0,8.6625,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
896,3,22.0,1,1,12.2875,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
897,3,14.0,0,0,9.2250,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
898,3,30.0,0,0,7.6292,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
899,2,26.0,1,1,29.0000,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
900,3,18.0,0,0,7.2292,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
